# Stock Price Comparison Tool

### Select multiple companies to compare their stock performance</h4>"
 
<mark>Use ctrl+click to select multiple companies</mark>

<!-- TODO:
- Add linked histogram to select part of the graph and display the returns profile
- Link all widgets to allow updating of start and end dates and add new tickers
- Tidy up css
- Add tab with optimial portfolio calculation
- Create user input section
-->

In [1]:
import datetime
import time
import warnings
from typing import Tuple

import calmap
import ffn
import ipywidgets as widgets
import pandas as pd

%load_ext autoreload
%autoreload 2

from content import Correlations, Drawdown, RebasedStockGraph, StockStatistics
from gui import Gui
from user_input import UserInputWidget

warnings.filterwarnings("ignore")

In [2]:
%matplotlib inline
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
stock_list = {
    "SNP500": "spy",
    "Apple": "aapl",
    "Microsoft": "msft",
    "Facebook": "fb",
    "Tesla": "tsla",
    "Google": "googl",
}

<IPython.core.display.Javascript object>

In [4]:
def get_data(
    companies: Tuple[str],
    start_date: str,
    end_date: str,
    stock_list: dict[str, str] = stock_list,
) -> pd.DataFrame:
    """Get data from Yahoo Finance endpoint using ffn"""
    time.sleep(1)  # avoid spamming
    tickers = [stock_list[company] for company in companies]
    return ffn.get(tickers, start=start_date, end=end_date)


def convert_ticker_to_company(
    ticker: str, stock_list: dict[str, str] = stock_list
) -> str:
    inverted_stock_list = {v: k for k, v in stock_list.items()}
    return inverted_stock_list[ticker]


def calc_yoy_growth(prices: pd.DataFrame, rolling_window: int = 28) -> pd.DataFrame:
    rebased_prices = prices.rebase()
    yoy_growth = rebased_prices.resample("D").sum().replace(0, method="ffill")
    yoy_growth = (
        yoy_growth.groupby([yoy_growth.index.day, yoy_growth.index.month])
        .pct_change()
        .dropna(axis=0)
    )
    return yoy_growth.rolling(window=rolling_window).mean().dropna()


def plot_calmap(yoy_growth: pd.Series, ticker):
    return calmap.calendarplot(
        yoy_growth[ticker],
        monthticks=1,
        daylabels="MTWTF",
        dayticks=[0, 2, 4],
        cmap="RdYlGn",
        linewidth=0.2,
        yearascending=False,
        yearlabel_kws=dict(color="#696969"),
        fig_kws=dict(figsize=(12, 8)),
    )

<IPython.core.display.Javascript object>

In [5]:
## Input Area
tickers = UserInputWidget(
    "tickers",
    widgets.SelectMultiple(
        options=stock_list.keys(),
        description="Stock",
        value=list(stock_list.keys())[:2],  # select first two
        disabled=False,
        layout=widgets.Layout(height="150px"),
    ),
)

start_date = UserInputWidget(
    "start_date",
    widgets.DatePicker(
        description="Start Date",
        value=datetime.datetime.today() - datetime.timedelta(days=3 * 365),
    ),
)

end_date = UserInputWidget(
    "end_date",
    widgets.DatePicker(description="End Date", value=datetime.datetime.today()),
)

get_stock_data_button = widgets.Button(
    description="Get data for selected stocks",
    disabled=False,
    button_style="info",
    tooltip="Click me",
    layout=widgets.Layout(width="auto", height="30px", margin="30px"),
)

<IPython.core.display.Javascript object>

In [6]:
## Graph
prices = get_data(tickers.widget.value, start_date.widget.value, end_date.widget.value)

<IPython.core.display.Javascript object>

In [7]:
rebased_stock_graph = RebasedStockGraph(
    "Rebased", "Rebased share price performance", prices
)

stock_statistics = StockStatistics(
    "Stock Statistics", "Statistics for each stock", prices
)

correlations = Correlations(
    "Correlations", "Correlations between selected stocks", prices
)

drawdown = Drawdown("Drawdown", "Drawdown periods", prices)

<IPython.core.display.Javascript object>

In [8]:
g = Gui(
    user_inputs=[tickers, start_date, end_date],
    action_button=get_stock_data_button,
    main=rebased_stock_graph,
    tabs=[stock_statistics, correlations, drawdown],
)

<IPython.core.display.Javascript object>

In [9]:
g.build()

<IPython.core.display.Javascript object>

Created by [EngineeringforDataScience](https://engineeringfordatascience.com/) ⚒️ | [Source Code](https://github.com/julian-west/e4ds-snippets/tree/voila/jupyter/voila)

In [10]:
# Portfolio optimisation

<IPython.core.display.Javascript object>